In [1]:
import re
import pandas as pd
import numpy as np

In [2]:
def parse_string_to_dict(s):
    # 먼저 key=value 쌍을 구분
    pairs = re.findall(r'(\w+)=(".*?"|\S+)', s)
    
    # 각 key=value 쌍을 딕셔너리에 저장
    parsed_dict = {}
    for key, value in pairs:
        # 따옴표 제거 및 값의 형태에 따라 변환

        if re.match(r'^-?\d+(\.\d+)?$', value):
            value = float(value) if '.' in value else int(value)
        elif value.startswith('"') and value.endswith('"'):
            value = list(value[1:-1].split(' '))
        parsed_dict[key] = value

    return parsed_dict

def parse_xyz(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()
    
    data = []
    start_idx = 0
    
    while start_idx < len(lines):
        num_atoms = int(lines[start_idx].strip())
        properties_line = lines[start_idx + 1].strip()
        pro_dict = parse_string_to_dict(properties_line)
        print(pro_dict)
        
        energy = float(properties_line.split('energy=')[1].split()[0])
        atom_type = []
        position = []
        force = []
        for i in range(start_idx + 2, start_idx + 2 + num_atoms):
            parts = lines[i].split()
            atom_type.append(parts[0])
            position.append([float(parts[1]), float(parts[2]), float(parts[3])])
            force.append([float(parts[4]), float(parts[5]), float(parts[6])])
        
        data.append({
            'e': energy,
            't': atom_type,
            'p': position,
            'f': force
        })
        start_idx += 2 + num_atoms
    
    return data

def reindexing(org, prd):
    # org 데이터의 첫 번째 p 값을 키로, 해당 인덱스를 값으로 하는 딕셔너리 생성
    org_map = {tuple(item['p'][0]): i for i, item in enumerate(org)}
    print(org_map)
    # prd 데이터를 재정렬할 리스트
    reordered_prd = [None] * len(prd)
    
    # prd 데이터에서 p 값을 찾아 org_map에 있는 인덱스로 재정렬
    for item in prd:
        p_value = tuple(item['p'][0])
        if p_value in org_map:
            reordered_prd[org_map[p_value]] = item
    
    # 재정렬된 prd 반환
    return reordered_prd
    

def make_df(data):
    d = []
    for i, item in enumerate(data):
        d.append({
            'ID': f'TEST_{i:04d}',
            'energy': item['e'],
            'forces': item['f']
        })
    df = pd.DataFrame(d)
    df['energy_uncertainty'] = 1
    cols = ['ID', 'energy', 'energy_uncertainty', 'forces']
    df = df[cols]
    return df


In [5]:
prd_data = parse_xyz('output.xyz')
prd_df = make_df(prd_data)

{'Lattice': ['11.07075023651123', '0.0', '0.0', '0.0', '11.07075023651123', '0.0', '0.0', '0.0', '11.07075023651123'], 'Properties': 'species:S:1:pos:R:3:energies:R:1:forces:R:3', 'original_dataset_index': 1500, 'energy': -935.9360961914062, 'stress': ['0.0244381595402956', '0.010940978303551674', '-0.009752786718308926', '0.010940978303551674', '0.016084428876638412', '0.005897898692637682', '-0.009752786718308926', '0.005897898692637682', '-0.007561351638287306'], 'pbc': ['F', 'F', 'F']}
{'Lattice': ['11.07075023651123', '0.0', '0.0', '0.0', '11.07075023651123', '0.0', '0.0', '0.0', '11.07075023651123'], 'Properties': 'species:S:1:pos:R:3:energies:R:1:forces:R:3', 'original_dataset_index': 1501, 'energy': -919.5879516601562, 'stress': ['0.010824857279658318', '-0.008262784220278263', '-0.006625918671488762', '-0.008262784220278263', '-0.01857743412256241', '0.019828271120786667', '-0.006625918671488762', '0.019828271120786667', '-0.029410813003778458'], 'pbc': ['F', 'F', 'F']}
{'Latt

In [6]:
prd_df.to_csv('sub.csv', index=False)

In [9]:
!tar -xf si6.zip -C /

results/silicon-tutorial/si/: Can't create '\\?\C:\results\silicon-tutorial\si': No such file or directory
results/silicon-tutorial/si/trainer.pth: Can't create '\\?\C:\results\silicon-tutorial\si\trainer.pth': No such file or directory
results/silicon-tutorial/si/last_model.pth: Can't create '\\?\C:\results\silicon-tutorial\si\last_model.pth': No such file or directory
results/silicon-tutorial/si/config.yaml: Can't create '\\?\C:\results\silicon-tutorial\si\config.yaml': No such file or directory
results/silicon-tutorial/si/metrics_initialization.csv: Can't create '\\?\C:\results\silicon-tutorial\si\metrics_initialization.csv': No such file or directory
results/silicon-tutorial/si/best_model.pth: Can't create '\\?\C:\results\silicon-tutorial\si\best_model.pth': No such file or directory
results/silicon-tutorial/si/metrics_epoch.csv: Can't create '\\?\C:\results\silicon-tutorial\si\metrics_epoch.csv': No such file or directory
results/silicon-tutorial/si/metrics_batch_train.csv: Can't 

In [19]:
import numpy as np
# 추가적인 3개의 서브미션 파일들이 있다고 가정하고 그 불확실성을 계산하여 추가하는 코드 작성

# sub1, sub2, sub3 파일이 있다고 가정하고 불러온다고 가정
file_paths = ['sub1.csv', 'sub2.csv', 'sub3.csv']

# 서브미션 파일들을 불러와 데이터프레임으로 변환
dfs = [pd.read_csv(file_path) for file_path in file_paths]

# 모델별 forces 값의 평균을 구한 후 이를 기반으로 불확실성 계산
# 여기서는 dfs 리스트에 각 모델(sub1, sub2, sub3)의 데이터프레임이 있다고 가정합니다.

# 예시로 3개의 서브미션 파일 데이터를 다시 처리
# 먼저 각 모델별 forces 평균을 구한 후, 이를 이용해 불확실성을 계산

# 모델 개수 J
J = len(dfs)

# 각 모델의 forces 데이터에서 평균을 계산하여 모델별 평균 forces 배열을 구함
mean_forces_per_model = [np.mean(np.stack(df['forces'].apply(lambda x: np.array(eval(x))).values), axis=1) for df in dfs]

# 성분별 평균 forces 계산 (모든 모델에 대해 평균)
mean_forces_overall = np.mean(np.stack(mean_forces_per_model), axis=0)

# 불확실성 계산: 각 모델의 평균 forces와 전체 평균 forces 간의 차이를 사용
diffs = np.stack([(mean_force - mean_forces_overall)**2 for mean_force in mean_forces_per_model])
diffs_mean = np.mean(diffs, axis=0) # 모델 mean
uncertainty = np.sqrt(np.mean(diffs_mean, axis=1) / 3)  # xyz mean

sub = pd.read_csv('sub1.csv')
sub['energy_uncertainty'] = uncertainty
sub.to_csv('sub.csv', index=False)

In [ ]:
df = pd.read_csv('sub.csv')

forces = np.stack(df['forces'].apply(lambda x: np.array(eval(x))).values)
mean_forces_lattice = np.mean(forces, axis=1) 
diffs = np.stack([(mean_force - mean_forces_lattice)**2 for mean_force in mean_forces_per_model])


sub['energy_uncertainty'] = uncertainty
sub.to_csv('sub.csv', index=False)